In [1]:
import pandas as pd
import json

In [2]:
elec16_file = "Data/elec16v2-Copy1.csv"
elec20_file = "Data/elec20v2-Copy1.csv"


In [3]:
election2020_data = pd.read_csv(elec20_file, low_memory=False)
election2016_data = pd.read_csv(elec16_file, low_memory=False)


In [4]:
states_file = "static/data/US_States.csv"
states_abbv= pd.read_csv(states_file,low_memory=False)

In [5]:
states_abbv.head(3)

,properties.name,state_abbv
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS


## Year 2020 Data Cleanup

In [6]:
election2020_data.head(2)

,year,office,party_simple,mode,votes,countyname,countyfips,candidatename,state_abbv,statefips,combinedfips
0,2020,US PRESIDENT,DEMOCRAT,ELECTION DAY,239,NaN,0,JOSEPH R BIDEN,AK,2,200000
1,2020,US PRESIDENT,REPUBLICAN,ELECTION DAY,466,NaN,0,DONALD J TRUMP,AK,2,200000


In [7]:
#Checking the row count on the data
election2020_data.count()

year             477324
office           477324
party_simple     477324
mode             477324
votes            477324
countyname       476158
countyfips       477324
candidatename    477324
state_abbv       477324
statefips        477324
combinedfips     477324
dtype: int64

In [8]:
#Cleaning up 2020 to remove the NAs from the county name
election2020_data_clean = election2020_data.query("countyfips != 0")

In [9]:
election2020_data_clean.count()

year             476158
office           476158
party_simple     476158
mode             476158
votes            476158
countyname       476158
countyfips       476158
candidatename    476158
state_abbv       476158
statefips        476158
combinedfips     476158
dtype: int64

In [10]:
election2020_data_clean["mode"].unique()

array(['TOTAL', 'ABSENTEE', 'PROVISIONAL', 'EARLY', 'ELECTION DAY',
       'ADVANCED VOTING', 'PRES', 'EARLY VOTING', '2ND ABSENTEE',
       'UOCAVA', 'MAIL', 'HAND COUNTED', 'ABSENTEE BY MAIL', 'ONE STOP',
       'MAIL BALLOTS', 'IN PERSON ABSENTEE', 'FAILSAFE',
       'FAILSAFE PROVISIONAL'], dtype=object)

In [11]:
#YEAR 2020 - USING THE CLEANED DATAFRAME TO SEPARATE DEMOCRATES FROM REPUBLICANS

rep_2020 = election2020_data_clean.loc[election2020_data_clean["party_simple"] == "REPUBLICAN",:]
demo_2020 = election2020_data_clean.loc[election2020_data_clean["party_simple"] == "DEMOCRAT",:]

In [12]:
grouped_rep_data_2020 = rep_2020.groupby(["state_abbv"])
grouped_rep_data_2020

In [13]:
rep_2020_votes_df= pd.DataFrame(grouped_rep_data_2020["votes"].sum())
rep_2020_votes_df.head(2) #total of 49 states

,votes
state_abbv,
AL,1441170
AR,760647


In [14]:
rep_2020_votes_df=rep_2020_votes_df.rename(columns={"votes":"RepTotalVotes"})

rep_2020_votes_df= rep_2020_votes_df.reset_index(drop=False)
rep_2020_votes_df.head(2)

,state_abbv,RepTotalVotes
0,AL,1441170
1,AR,760647


In [15]:
grouped_demo_data_2020 = demo_2020.groupby(["state_abbv"])
grouped_demo_data_2020

In [16]:
demo_2020_votes_df= pd.DataFrame(grouped_demo_data_2020["votes"].sum())
demo_2020_votes_df.head(2) #total of 47 states

,votes
state_abbv,
AL,849624
AR,423932


In [17]:
demo_2020_votes_df=demo_2020_votes_df.rename(columns={"votes":"DemoTotalVotes"})
#demo_2020_votes_take2=demo_2020_votes_df.rename(columns={"votes":"Demo 2020"})
demo_2020_votes_df= demo_2020_votes_df.reset_index(drop=False)
demo_2020_votes_df.head(2)

,state_abbv,DemoTotalVotes
0,AL,849624
1,AR,423932


#TABLING THE MODE FOR NOW.. NOT CLEANING IT YET
#value_count only gives the number of row entry, but i need the total votes for each category

grouped_rep_data_2020["mode"].value_counts() #total of 101

In [18]:
len(rep_2020_votes_df)

49

In [19]:
len(demo_2020_votes_df)

47

In [20]:
# votes_2020_take2 = rep_2020_votes_take2.merge(demo_2020_votes_take2, on="state_abbv",how="left")
# votes_2020_take2.head(2)

In [21]:
votes_2020 = rep_2020_votes_df.merge(demo_2020_votes_df, on="state_abbv",how="left")
votes_2020.head(2)

,state_abbv,RepTotalVotes,DemoTotalVotes
0,AL,1441170,849624.0
1,AR,760647,423932.0


In [22]:
final_votes_2020 = votes_2020.merge(states_abbv, on="state_abbv",how="left")
final_votes_2020

,state_abbv,RepTotalVotes,DemoTotalVotes,properties.name
0,AL,1441170,849624.0,Alabama
1,AR,760647,423932.0,Arkansas
2,AZ,1661686,1672143.0,Arizona
3,CA,6006518,11110639.0,California
4,CO,1364607,1804352.0,Colorado
5,CT,714697,1080680.0,Connecticut
6,DC,18586,317323.0,District of Columbia
7,DE,200603,296268.0,Delaware
8,FL,5668716,5297036.0,Florida
9,GA,2461837,2474507.0,Georgia


In [23]:
votes_2020["state_abbv"].dtype

dtype('O')

In [24]:
votes_2020=votes_2020.fillna(0)
votes_2020.head(2)

,state_abbv,RepTotalVotes,DemoTotalVotes
0,AL,1441170,849624.0
1,AR,760647,423932.0


## Year 2016 Data Cleanup

In [25]:
election2016_data.head(2)

,year,office,party,mode,votes,countyname,countyfips,candidatename,state_abbv,statefips,combinedfips
0,2016,US President,democratic,election day,135,Autauga County,1001,Hillary Clinton,AL,1,101001
1,2016,US President,republican,election day,218,Autauga County,1001,Donald Trump,AL,1,101001


In [26]:
#Checking the row count on the data
election2016_data.count()

year             460232
office           460232
party            460232
mode             460232
votes            460232
countyname       458512
countyfips       460232
candidatename    460232
state_abbv       460232
statefips        460232
combinedfips     460232
dtype: int64

In [27]:
election2016_data_clean = election2016_data.dropna(how="any")

In [28]:
election2016_data_clean.count()

year             458512
office           458512
party            458512
mode             458512
votes            458512
countyname       458512
countyfips       458512
candidatename    458512
state_abbv       458512
statefips        458512
combinedfips     458512
dtype: int64

In [29]:
#YEAR 2016 - USING THE CLEANED DATAFRAME TO SEPARATE DEMOCRATES FROM REPUBLICANS

rep_2016 = election2016_data_clean.loc[election2016_data_clean["party"] == "republican",:]
demo_2016 = election2016_data_clean.loc[election2016_data_clean["party"] == "democratic",:]

In [30]:
#NEED TO DO A GROUP BY STATE FOR EACH PARTY

grouped_rep_data_2016 = rep_2016.groupby(["state_abbv"])
rep_2016_votes_df= pd.DataFrame(grouped_rep_data_2016["votes"].sum())

grouped_demo_data_2016 = demo_2016.groupby(["state_abbv"])
demo_2016_votes_df= pd.DataFrame(grouped_demo_data_2016["votes"].sum())

In [31]:
rep_2016_votes_df=rep_2016_votes_df.rename(columns={"votes":"RepTotalVotes"})

rep_2016_votes_df= rep_2016_votes_df.reset_index(drop=False)


rep_2016_votes_df.head(2)

,state_abbv,RepTotalVotes
0,AL,1966420
1,AR,684872


In [32]:
demo_2016_votes_df=demo_2016_votes_df.rename(columns={"votes":"DemoTotalVotes"})

demo_2016_votes_df= demo_2016_votes_df.reset_index(drop=False)


demo_2016_votes_df.head(2)

,state_abbv,DemoTotalVotes
0,AL,1240976
1,AR,380494


In [33]:
len(demo_2016_votes_df)

47

In [34]:
len(rep_2016_votes_df)

49

In [35]:
votes_2016 = rep_2016_votes_df.merge(demo_2016_votes_df, on="state_abbv",how="left")
votes_2016= votes_2016.fillna(0)
votes_2016.head(2)

,state_abbv,RepTotalVotes,DemoTotalVotes
0,AL,1966420,1240976.0
1,AR,684872,380494.0


In [36]:
final_votes_2016 = votes_2016.merge(states_abbv, on="state_abbv",how="left")
final_votes_2016

,state_abbv,RepTotalVotes,DemoTotalVotes,properties.name
0,AL,1966420,1240976.0,Alabama
1,AR,684872,380494.0,Arkansas
2,AZ,1252389,1161165.0,Arizona
3,CA,4483799,8753777.0,California
4,CO,1202495,1338870.0,Colorado
5,CT,673215,897572.0,Connecticut
6,DC,12723,282830.0,District of Columbia
7,DE,185127,235603.0,Delaware
8,FL,4617879,4504974.0,Florida
9,GA,5494,1434.0,Georgia


In [37]:
#Converting US State Json to Dataframe, then merge with voting data

with open('static/data/us-states_2020.json','r') as f:
    data = json.loads(f.read())
    
#Flatten data    
converted_json = pd.json_normalize(data, record_path=['features'])
converted_json

,type,id,properties.name,properties.density,geometry.type,geometry.coordinates
0,Feature,01,Alabama,94.650,Polygon,"[[[-87.359296, 35.00118], [-85.606675, 34.9847..."
1,Feature,02,Alaska,1.264,MultiPolygon,"[[[[-131.602021, 55.117982], [-131.569159, 55...."
2,Feature,04,Arizona,57.050,Polygon,"[[[-109.042503, 37.000263], [-109.04798, 31.33..."
3,Feature,05,Arkansas,56.430,Polygon,"[[[-94.473842, 36.501861], [-90.152536, 36.496..."
4,Feature,06,California,241.700,Polygon,"[[[-123.233256, 42.006186], [-122.378853, 42.0..."
5,Feature,08,Colorado,49.330,Polygon,"[[[-107.919731, 41.003906], [-105.728954, 40.9..."
6,Feature,09,Connecticut,739.100,Polygon,"[[[-73.053528, 42.039048], [-71.799309, 42.022..."
7,Feature,10,Delaware,464.300,Polygon,"[[[-75.414089, 39.804456], [-75.507197, 39.683..."
8,Feature,11,District of Columbia,10065.000,Polygon,"[[[-77.035264, 38.993869], [-76.909294, 38.895..."
9,Feature,12,Florida,353.400,Polygon,"[[[-85.497137, 30.997536], [-85.004212, 31.003..."


In [38]:
#Join existing GeoCoordinates to 2016 Data
votes_2016_geojson = converted_json.merge(final_votes_2016, on="properties.name",how="left")
votes_2016_geojson=votes_2016_geojson.rename(columns={"state_abbv":"properties.state_abbv",
                                                    "RepTotalVotes":"properties.RepTotalVotes",
                                                   "DemoTotalVotes":"properties.DemoTotalVotes"})
votes_2016_geojson_final= votes_2016_geojson[["type","id","properties.name","properties.state_abbv",
                                        "properties.RepTotalVotes","properties.DemoTotalVotes","properties.density",
                                       "geometry.type","geometry.coordinates"]]
#votes_2016_geojson_final

In [39]:
#Join existing GeoCoordinates to 2020 Data
votes_2020_geojson = converted_json.merge(final_votes_2020, on="properties.name",how="left")

votes_2020_geojson=votes_2020_geojson.rename(columns={"state_abbv":"properties.state_abbv",
                                                    "RepTotalVotes":"properties.RepTotalVotes",
                                                   "DemoTotalVotes":"properties.DemoTotalVotes"})
votes_2020_geojson_final= votes_2020_geojson[["type","id","properties.name","properties.state_abbv",
                                        "properties.RepTotalVotes","properties.DemoTotalVotes","properties.density",
                                       "geometry.type","geometry.coordinates"]]

votes_2020_geojson_final


,type,id,properties.name,properties.state_abbv,properties.RepTotalVotes,properties.DemoTotalVotes,properties.density,geometry.type,geometry.coordinates
0,Feature,01,Alabama,AL,1441170.0,849624.0,94.650,Polygon,"[[[-87.359296, 35.00118], [-85.606675, 34.9847..."
1,Feature,02,Alaska,NaN,NaN,NaN,1.264,MultiPolygon,"[[[[-131.602021, 55.117982], [-131.569159, 55...."
2,Feature,04,Arizona,AZ,1661686.0,1672143.0,57.050,Polygon,"[[[-109.042503, 37.000263], [-109.04798, 31.33..."
3,Feature,05,Arkansas,AR,760647.0,423932.0,56.430,Polygon,"[[[-94.473842, 36.501861], [-90.152536, 36.496..."
4,Feature,06,California,CA,6006518.0,11110639.0,241.700,Polygon,"[[[-123.233256, 42.006186], [-122.378853, 42.0..."
5,Feature,08,Colorado,CO,1364607.0,1804352.0,49.330,Polygon,"[[[-107.919731, 41.003906], [-105.728954, 40.9..."
6,Feature,09,Connecticut,CT,714697.0,1080680.0,739.100,Polygon,"[[[-73.053528, 42.039048], [-71.799309, 42.022..."
7,Feature,10,Delaware,DE,200603.0,296268.0,464.300,Polygon,"[[[-75.414089, 39.804456], [-75.507197, 39.683..."
8,Feature,11,District of Columbia,DC,18586.0,317323.0,10065.000,Polygon,"[[[-77.035264, 38.993869], [-76.909294, 38.895..."
9,Feature,12,Florida,FL,5668716.0,5297036.0,353.400,Polygon,"[[[-85.497137, 30.997536], [-85.004212, 31.003..."


In [41]:
#Simplier approach
def df_to_formatted_json(df, sep="."):
    """
    The opposite of json_normalize
    """
    result = []
    for idx, row in df.iterrows():
        parsed_row = {}
        for col_label,v in row.items():
            keys = col_label.split(".")

            current = parsed_row
            for i, k in enumerate(keys):
                if i==len(keys)-1:
                    current[k] = v
                else:
                    if k not in current.keys():
                        current[k] = {}
                    current = current[k]
        # save
        result.append(parsed_row)
    return result

In [42]:
geo2016= df_to_formatted_json(votes_2016_geojson_final,sep=".")

In [43]:
geo2020= df_to_formatted_json(votes_2020_geojson_final,sep=".")

In [44]:
#Creating the GeoJson for Year 2020;Variable assigned updated in VS code

geojson_2020 = open("static/data/geojson_2020_js.js", "w")
geojson_2020.write(str(geo2020))
geojson_2020.close()

In [45]:
#Creating the GeoJson for Year 2016;Variable assigned updated in VS code
geojson_2016 = open("static/data/geojson_2016_js.js", "w")
geojson_2016.write(str(geo2016))
geojson_2016.close()